# Asset Class Trend Following 策略實作 (含停損機制最佳化)

本筆記本實作了「個股V2.docx」與「reproduce_strategy-停損測試.md」中要求的策略。該策略基於 Trend Following 與 Momentum 指標，並加入停損機制，透過螞蟻演算法 (ACO) 進行參數最佳化。

## 策略邏輯摘要：
- **資產池**: 122 檔個股
- **進場條件**: 價格 > SMA 且 ROC > 0
- **排名方式**: 符合條件標的依 ROC (動能) 由高到低排名，取前 3 名
- **再平衡**: 每 5 個交易日評估一次
- **執行**: T 日訊號，T+1 日收盤價成交
- **持倉管理**: 若原有持股仍在 Top 3 則「續抱」；若觸發停損則出場且該期不再買入
- **最佳化**: 使用 ACO 尋找最佳停損參數

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from datetime import datetime
import xlsxwriter

# ==========================================
# 1. 參數設定區域
# ==========================================
DATA_PATH = '0205/data.xlsx'
INITIAL_CAPITAL = 30_000_000
MAX_HOLDINGS = 3
REBALANCE_INTERVAL = 5
SMA_PERIOD = 30
ROC_PERIOD = 100

In [ ]:
# ==========================================
# 2. 資料載入與清理
# ==========================================
def load_and_clean_data(filepath):
    # 讀取 Excel (多層標頭)
    df = pd.read_excel(filepath, header=[0, 1], index_col=0)
    df.index = pd.to_datetime(df.index)
    
    # 依 Ticker 去重 (Level 0 為 Ticker)
    df = df.loc[:, ~df.columns.get_level_values(0).duplicated()]
    
    # 資料清理：ffill (中間空值) 然後 bfill (起始空值)
    df = df.ffill().bfill()
    
    ticker_to_name = dict(zip(df.columns.get_level_values(0), df.columns.get_level_values(1)))
    df.columns = df.columns.get_level_values(0)
    
    return df, ticker_to_name

prices_df, ticker_to_name = load_and_clean_data(DATA_PATH)
print(f"資料載入完成，共 {len(prices_df.columns)} 檔標的，時間範圍: {prices_df.index[0].date()} 至 {prices_df.index[-1].date()}")

In [ ]:
# ==========================================
# 3. 策略回測核心邏輯
# ==========================================
def run_backtest(prices_df, ticker_to_name, sma_p, roc_p, 
                 stop_loss_type=None, stop_loss_param=None, 
                 capital=INITIAL_CAPITAL, max_holdings=MAX_HOLDINGS, 
                 rebalance_interval=REBALANCE_INTERVAL):
    
    # 計算技術指標
    sma = prices_df.rolling(window=sma_p).mean()
    roc = prices_df.pct_change(periods=roc_p)
    
    ma_stop_series = None
    if stop_loss_type == 'MA':
        ma_stop_series = prices_df.rolling(window=int(stop_loss_param)).mean()
    
    dates = prices_df.index
    equity = pd.Series(index=dates, data=0.0)
    cash = capital
    holdings = {} # ticker -> {'shares': n, 'buy_price': p, 'max_price': p}
    
    events = []
    equity_hold_log = []
    
    # 待執行訂單 (T 日觸發，T+1 執行)
    pending_exit = {} 
    pending_rebalance = None
    stopped_out_this_period = set()
    
    start_idx = max(sma_p, roc_p)
    if stop_loss_type == 'MA': 
        start_idx = max(start_idx, int(stop_loss_param))
    signal_days = [i for i in range(start_idx, len(dates), rebalance_interval)]
    
    for i in range(len(dates)):
        current_date = dates[i]
        
        # A. 執行 T+1 交易 (開盤/收盤，此處採用收盤價)
        # 1. 停損出場
        for t, reason in list(pending_exit.items()):
            if t in holdings:
                info = holdings.pop(t)
                sell_p = prices_df.loc[current_date, t]
                profit = (sell_p - info['buy_price']) * info['shares']
                ret = (sell_p / info['buy_price']) - 1
                cash += info['shares'] * sell_p
                events.append({
                    '日期': current_date, 'Ticker': t, '動作': '停損', 
                    '價格': sell_p, '股數': info['shares'], '損益': profit, '報酬率': ret,
                    '原因': reason, '說明': f"觸發{reason}，於 T+1 日收盤價 {sell_p:.2f} 出場"
                })
        pending_exit = {}
        
        # 2. 再平衡執行
        if pending_rebalance is not None:
            targets, sig_roc = pending_rebalance
            current_set = set(holdings.keys())
            target_set = set(targets)
            to_sell = current_set - target_set
            to_keep = current_set & target_set
            to_buy = target_set - current_set
            
            for t in to_sell:
                info = holdings.pop(t)
                sell_p = prices_df.loc[current_date, t]
                profit = (sell_p - info['buy_price']) * info['shares']
                ret = (sell_p / info['buy_price']) - 1
                cash += info['shares'] * sell_p
                events.append({
                    '日期': current_date, 'Ticker': t, '動作': '剔除', 
                    '價格': sell_p, '股數': info['shares'], '損益': profit, '報酬率': ret,
                    '原因': '跌出排名', '說明': f"不再符合前三名，賣出價格 {sell_p:.2f}"
                })
            
            for t in to_keep:
                events.append({
                    '日期': current_date, 'Ticker': t, '動作': '續抱', 
                    '價格': prices_df.loc[current_date, t], '股數': holdings[t]['shares'],
                    '原因': '續抱', '說明': f"資產 {t} 續抱，動能值 {sig_roc[t]:.4f}"
                })
                
            if to_buy:
                buy_cash = cash / len(to_buy)
                for t in to_buy:
                    p = prices_df.loc[current_date, t]
                    s = buy_cash // p
                    if s > 0:
                        holdings[t] = {'shares': s, 'buy_price': p, 'max_price': p}
                        cash -= s * p
                        events.append({
                            '日期': current_date, 'Ticker': t, '動作': '新增', 
                            '價格': p, '股數': s, '原因': '新增', 
                            '說明': f"新進前三名，買入價格 {p:.2f}"
                        })
            
            equity_hold_log.append({
                '日期': current_date, '持股數': len(holdings), 
                '明細': ", ".join(holdings.keys()), 
                '總市值': cash + sum(h['shares'] * prices_df.loc[current_date, tk] for tk, h in holdings.items())
            })
            pending_rebalance = None
            stopped_out_this_period = set() # 重置本期停損清單
            
        # B. 權益更新 (交易執行後)
        port_val = sum(info['shares'] * prices_df.loc[current_date, t] for t, info in holdings.items())
        equity.iloc[i] = cash + port_val
        
        # C. 每日檢查 (T 日)
        # 1. 停損檢查
        for t in list(holdings.keys()):
            current_p = prices_df.loc[current_date, t]
            if current_p > holdings[t]['max_price']: 
                holdings[t]['max_price'] = current_p
            
            triggered = False; reason = ""
            if stop_loss_type == 'Peak':
                if current_p < holdings[t]['max_price'] * (1 - stop_loss_param):
                    triggered = True; reason = "回落停損"
            elif stop_loss_type == 'MA':
                if current_p < ma_stop_series.loc[current_date, t]:
                    triggered = True; reason = "均線停損"
            
            if triggered:
                pending_exit[t] = reason
                stopped_out_this_period.add(t)

        # 2. 再平衡檢查
        if i in signal_days:
            curr_p = prices_df.iloc[i]
            curr_sma = sma.iloc[i]
            curr_roc = roc.iloc[i]
            
            eligible = curr_roc[(curr_p > curr_sma) & (curr_roc > 0)].sort_values(ascending=False)
            # 排除本期已停損標的
            eligible = eligible[~eligible.index.isin(stopped_out_this_period)]
            
            targets = eligible.head(max_holdings).index.tolist()
            pending_rebalance = (targets, curr_roc)

    return equity, events, equity_hold_log

In [ ]:
# ==========================================
# 4. 績效分析工具
# ==========================================
def calculate_metrics(equity):
    if equity.iloc[-1] == equity.iloc[0]: return 0, 0, 0, pd.Series(0, index=equity.index)
    total_ret = (equity.iloc[-1] / equity.iloc[0]) - 1
    days = (equity.index[-1] - equity.index[0]).days
    cagr = (1 + total_ret)**(365.25/days) - 1
    roll_max = equity.cummax()
    dd = (equity - roll_max) / roll_max
    mdd = dd.min()
    calmar = cagr / abs(mdd) if mdd != 0 else 0
    return cagr, mdd, calmar, dd

In [ ]:
# ==========================================
# 5. 螞蟻演算法 (ACO) 最佳化實作
# ==========================================
def aco_optimize(prices_df, ticker_to_name, stop_loss_type, param_range, iterations=5, n_ants=3):
    n_params = len(param_range)
    pheromones = np.ones(n_params)
    best_param = None
    best_calmar = -np.inf
    
    print(f"開始 {stop_loss_type} 停損參數最佳化...")
    for i in range(iterations):
        probs = pheromones / pheromones.sum()
        ants_idx = np.random.choice(n_params, size=n_ants, p=probs)
        calmars = []
        for idx in ants_idx:
            param = param_range[idx]
            eq, _, _ = run_backtest(prices_df, ticker_to_name, 30, 100, stop_loss_type, param)
            _, _, calmar, _ = calculate_metrics(eq)
            calmars.append(calmar)
            if calmar > best_calmar:
                best_calmar = calmar
                best_param = param
        
        pheromones *= 0.8 # 蒸發
        for idx, clm in zip(ants_idx, calmars):
            pheromones[idx] += max(0, clm)
        print(f"  迭代 {i+1}: 目前最佳 Calmar={best_calmar:.2f} (參數={best_param})")
    return best_param, best_calmar

# 執行最佳化
peak_range = np.arange(0.02, 0.101, 0.01)
best_peak_p, best_peak_c = aco_optimize(prices_df, ticker_to_name, 'Peak', peak_range)

ma_range = np.arange(3, 16, 1)
best_ma_p, best_ma_c = aco_optimize(prices_df, ticker_to_name, 'MA', ma_range)

if best_ma_c >= best_peak_c:
    BEST_STOP_LOSS_TYPE = 'MA'
    BEST_STOP_LOSS_PARAM = best_ma_p
    FINAL_CALMAR = best_ma_c
else:
    BEST_STOP_LOSS_TYPE = 'Peak'
    BEST_STOP_LOSS_PARAM = best_peak_p
    FINAL_CALMAR = best_peak_c

print(f"\nACO 搜尋完成。最終選擇: {BEST_STOP_LOSS_TYPE} 停損, 參數: {BEST_STOP_LOSS_PARAM}, Calmar: {FINAL_CALMAR:.2f}")

In [ ]:
# ==========================================
# 6. 參數高原表 (SMA vs Calmar, ROC=100)
# ==========================================
plateau_data = []
sma_list = [10, 20, 30, 40, 50, 60, 80, 100, 120, 150, 200]
for s in sma_list:
    eq, _, _ = run_backtest(prices_df, ticker_to_name, s, 100)
    c, m, cl, _ = calculate_metrics(eq)
    plateau_data.append({'SMA': s, 'CAGR': f"{c:.2%}", 'MaxDD': f"{m:.2%}", 'Calmar': round(cl, 2)})

plateau_df = pd.DataFrame(plateau_data)
display(plateau_df)

In [ ]:
# ==========================================
# 7. 最終策略執行與結果產出
# ==========================================
final_equity, final_events, final_hold_log = run_backtest(prices_df, ticker_to_name, 30, 100, BEST_STOP_LOSS_TYPE, BEST_STOP_LOSS_PARAM)
cagr, mdd, calmar, drawdown = calculate_metrics(final_equity)

plt.figure(figsize=(12, 6))
plt.plot(final_equity, label='Equity Curve')
plt.fill_between(final_equity.index, final_equity, final_equity.cummax(), color='red', alpha=0.3, label='Drawdown')
plt.title(f'Final Strategy (SMA=30, ROC=100, {BEST_STOP_LOSS_TYPE}={BEST_STOP_LOSS_PARAM})')
plt.legend()
plt.show()

with pd.ExcelWriter('strategy_results.xlsx', engine='xlsxwriter') as writer:
    pd.DataFrame(final_events).to_excel(writer, sheet_name='Trades', index=False)
    pd.DataFrame({'日期': final_equity.index, 'Equity': final_equity.values, 'Drawdown': drawdown.values}).to_excel(writer, sheet_name='Equity_Curve', index=False)
    pd.DataFrame(final_hold_log).to_excel(writer, sheet_name='Equity_Hold', index=False)
    
    closed_trades = [e for e in final_events if e['動作'] in ['剔除', '停損']]
    win_rate = sum(1 for t in closed_trades if t['損益'] > 0) / len(closed_trades) if closed_trades else 0
    
    summary_df = pd.DataFrame({
        '指標': ['SMA 參數', 'ROC 參數', '最佳停損機制', '最佳停損參數', 'CAGR', 'MaxDD', 'Calmar Ratio', '勝率'],
        '數值': [30, 100, BEST_STOP_LOSS_TYPE, BEST_STOP_LOSS_PARAM, f"{cagr:.2%}", f"{mdd:.2%}", round(calmar, 2), f"{win_rate:.2%}"]
    })
    summary_df.to_excel(writer, sheet_name='Summary', index=False)
    plateau_df.to_excel(writer, sheet_name='Plateau_Table', index=False)
    
    workbook = writer.book
    worksheet = writer.sheets['Equity_Curve']
    chart = workbook.add_chart({'type': 'line'})
    chart.add_series({
        'name': 'Equity',
        'categories': ['Equity_Curve', 1, 0, len(final_equity), 0],
        'values': ['Equity_Curve', 1, 1, len(final_equity), 1],
    })
    worksheet.insert_chart('E2', chart)

print("所有結果已儲存至 strategy_results.xlsx")